In [ ]:
import networkx as nx
from itertools import combinations
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import fileDefinitions as fd
import utils.helper_methods as helper_methods
from networkx.algorithms import bipartite
import matplotlib.pyplot as plt
import time
import random
from networkx.algorithms import link_prediction
from networkx.algorithms.assortativity import attribute_assortativity_coefficient
import math

In [ ]:

try:
    gph = nx.read_gexf("../graphs/v3/contribConnected.gexf")
    #helper_methods.logData("graph file found at " + str())
    #print("graph file found at " + str())
    print(gph)
except Exception as e:
    helper_methods.logData(e)
    gph = nx.Graph()
    #helper_methods.logData("new graph file generated")

In [ ]:
B = gph

# users = []
# issues = []
# for n in B:
#     if(n == 0):
#         users.append(n)
#     else:
#         issues.append(n)
users = [n for n, d in B.nodes(data=True) if d["bipartite"] == 0]
issues = [n for n, d in B.nodes(data=True) if d["bipartite"] == 1]


In [ ]:
users_matrix = nx.bipartite.biadjacency_matrix(B, row_order=users)
issues_matrix = nx.bipartite.biadjacency_matrix(B, row_order=issues)



In [ ]:
def weighted_projected_graph(B, nodes, weight_function=None):
    # Create a new graph for the weighted projection
    W = nx.Graph()

    # Get the nodes from one bipartite node set
    if nodes is None:
        nodes = set(B)

    # Get the weights for the nodes
    if weight_function is None:
        weight_function = lambda x, y, r: len(r)

    # For each pair of nodes in the same bipartite node set
    for u in nodes:
        for v in nodes:
            if u == v:
                continue
            neighbors_u = set(B[u])
            neighbors_v = set(B[v])

            # Find the shared neighbors of u and v
            shared_neighbors = neighbors_u & neighbors_v

            # Calculate the similarity probability
            weight = weight_function(u, v, shared_neighbors)

            # Add the weighted edge to the new graph
            if weight > 0:
                W.add_edge(u, v, weight=weight)

    return W

In [ ]:
def weighted_projected_graph(C_sub, usersC, issuesC):
    X_nodes = users
    Y_nodes = issues
    P = bipartite.weighted_projected_graph(C_sub, X_nodes)
    return P

In [ ]:
# Define a weight function for the shared neighbors
def weight_function(u, v, shared_neighbors):
    return len(shared_neighbors)

# Call the weighted_projected_graph function with the bipartite graph and the weight function
# W = weighted_projected_graph(C_sub, nodes=C_sub.nodes, weight_function=weight_function)
W = weighted_projected_graph(B, users, weight_function)

# Print the weighted projected graph
print(W.edges(data=True))

In [ ]:
def removeEdges(graph):
    g = nx.Graph()
    initial = graph.number_of_edges()
    issuesList = random.sample(issues, 2000)
    for i in issuesList:
        for j in list(graph.neighbors(i)):
            if graph.nodes[j]["bipartite"] == 0:
                g.add_node(i, bipartite=1)
                g.add_node(j, bipartite=0)
                g.add_edge(i,j)
                graph.remove_edge(i,j)
    final = graph.number_of_edges()
    print(initial - final)
    return g,graph

In [ ]:
G = B
og, test = removeEdges(G)

In [ ]:
pred_graph = weighted_projected_graph(og)

pred_list = pred_graph.edges()

In [ ]:
common_edges = []

for edge1 in pred_list:
    for edge2 in og_list:
        sorted_edge1 = tuple(sorted(edge1))
        sorted_edge2 = tuple(sorted(edge2))
        if sorted_edge1 == sorted_edge2:
            common_edges.append(edge1)
            break